# Mapper

In [60]:
#Install Library
!pip install -U giotto-tda

In [61]:
# Import all necessary modules
import pandas as pd
import numpy as np
from gtda.time_series import SingleTakensEmbedding
import matplotlib.pyplot as plt
from gtda.plotting import plot_point_cloud, plot_heatmap
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PersistenceEntropy
from sklearn.cluster import DBSCAN, KMeans, SpectralClustering, Birch, AgglomerativeClustering, MeanShift
from scipy import stats
from gtda.mapper import CubicalCover, make_mapper_pipeline, Eccentricity, Entropy, Projection, plot_static_mapper_graph,plot_interactive_mapper_graph


In [62]:
dbcd = pd.read_csv('/content/DBCD.csv')
dbcd

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A4913,A4914,A4915,A4916,A4917,A4918,A4919,A4920,A4921,A4922
0,12.997042,0,1,-0.818530,-0.267032,0.022596,-0.230246,0.198007,-0.509936,0.528540,...,0.000342,-0.148672,0.010020,-0.243390,-0.204327,-0.071979,0.009369,-0.028286,0.619705,-0.393669
1,11.157084,0,1,-0.391394,-0.309574,-0.060363,-0.163607,-0.173488,-0.033876,-0.091487,...,-0.158064,-0.020969,-0.092431,0.170701,-0.293380,-0.125255,-0.051292,0.046259,-0.061444,0.043108
2,10.138080,0,1,-0.624233,-0.119856,-0.235873,-0.174607,0.252850,-0.125016,-0.127058,...,-0.025513,0.292308,-0.116782,0.176038,-0.349162,-0.167774,-0.064431,0.095964,0.167486,0.545882
3,8.802201,0,1,-0.418720,0.161049,-0.329546,-0.023469,0.214949,0.029675,0.040544,...,-0.175852,0.137609,-0.366299,0.005222,-0.185165,0.193407,-0.185198,-0.042248,-0.103990,0.189081
4,10.294064,0,1,-0.810630,-0.536417,-0.211069,-0.200584,0.286504,-0.079759,-0.149976,...,-0.258462,-0.213575,-0.005606,0.091120,-0.104243,-0.161762,0.119698,0.030638,-0.762080,-0.363031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,8.424483,0,1,0.160275,0.228554,-0.300466,0.056553,0.022750,0.554721,0.067238,...,-0.275465,0.258161,0.661894,-0.085457,0.326344,-0.327657,-0.277535,0.168016,-0.087625,-0.040061
291,10.031029,0,1,0.261673,0.399931,-0.031858,0.211332,0.088976,0.404415,0.236193,...,0.045372,-0.415103,0.075576,-0.218558,0.138249,-0.007740,0.210111,-0.007115,1.298576,-0.404783
292,7.378547,0,1,-0.521894,-0.280549,-0.314779,0.132494,-0.213776,0.326215,-0.309374,...,-0.538544,-0.002992,0.206624,-0.022945,-0.107030,-0.443997,0.030036,-0.205376,-0.937327,-0.441240
293,6.754305,0,1,-0.013781,0.241821,0.113072,0.035607,-0.021082,0.276716,0.008162,...,-0.060761,0.047741,-0.033890,0.219354,-0.013937,0.158244,0.215542,0.178013,0.011729,-0.053175


In [63]:
dbcd_sort = dbcd.sort_values(by=['A1'])
del dbcd_sort['A2']
del dbcd_sort['A3']
dbcd_sort

,A1,A4,A5,A6,A7,A8,A9,A10,A11,A12,...,A4913,A4914,A4915,A4916,A4917,A4918,A4919,A4920,A4921,A4922
176,0.054812,0.019935,-0.043177,0.066085,-0.033071,0.163610,-0.175697,-0.034097,-0.057761,0.177973,...,-0.025966,0.035657,-0.102352,0.123709,-0.231527,0.175750,0.092046,-0.094414,-0.759283,0.387694
138,0.711861,0.937235,-0.244684,-0.116267,0.054862,-0.081377,0.051549,-0.083982,-0.025607,-0.678028,...,0.272174,0.141231,0.227815,-0.134530,0.509213,-0.075797,-0.105237,-0.143826,-0.803545,-0.287106
264,0.974736,-1.447581,-0.549368,-0.247172,-0.764167,-0.176743,-0.112853,0.054949,-0.076646,-0.298745,...,0.041889,0.191530,0.120646,-0.621534,0.536037,0.082599,-0.045721,-0.177401,-0.384109,-0.484176
177,1.073222,-0.199231,-0.136359,0.015944,0.030468,0.034441,0.117780,-0.185927,0.151229,-0.795187,...,-0.058601,0.163632,-0.081284,-0.218824,0.121970,-0.349587,-0.055202,-0.551755,-0.513010,-1.074615
143,1.316982,-0.303693,0.520857,-0.336211,-0.076130,-0.044480,-0.055277,-0.155827,-0.095284,-0.770644,...,0.009139,0.017278,-0.136529,-0.258067,-0.225903,-0.298038,-0.132905,-0.075359,-0.186038,-0.176791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,17.487091,-0.001087,0.181731,0.296670,-0.190403,-0.218437,-0.065673,-0.066405,0.184229,0.059148,...,0.147208,0.044413,-0.163085,0.099709,-0.153744,0.171604,-0.010180,0.118427,0.055516,-0.581992
69,17.574082,0.355349,-0.142103,-0.096278,0.007605,0.269356,-0.143557,0.424226,0.108667,-0.228418,...,0.103023,0.299938,0.003238,-0.133896,-0.178659,-0.066942,0.118276,0.085570,-0.787513,0.165153
73,17.659092,-0.318378,-0.003260,-0.247697,-0.092754,-0.132286,-0.181909,-0.342291,-0.199297,-0.106317,...,0.040297,-0.180987,-0.162543,0.355304,-0.030736,-0.251088,-0.082230,0.051855,-0.047637,0.027934
261,18.081076,-0.312718,-0.283437,0.525605,-0.357977,-0.008689,0.433591,-0.203122,-0.524636,-0.605653,...,-0.200713,-0.065145,0.173042,-0.613212,0.509196,0.007895,0.337673,-0.047114,0.037515,0.095523


In [90]:
# Define filter function – can be any scikit-learn transformer
filter_func = Projection(columns=['A1'])


# Define cover
cover = CubicalCover(n_intervals=6, overlap_frac=0.45)
# Choose clustering algorithm – default is DBSCAN
clusterer = AgglomerativeClustering(n_clusters=2)


# Configure parallelism of clustering step
n_jobs = 1

# Initialise pipeline
pipe = make_mapper_pipeline(
    filter_func=filter_func,
    cover=cover,
    clusterer=clusterer,
    verbose=False,
    n_jobs=n_jobs
)

fig = plot_static_mapper_graph(pipe, dbcd_sort, color_data=dbcd_sort, color_features=filter_func)
fig.show(config={'scrollZoom': True})

In [91]:
graph = pipe.fit_transform(dbcd_sort)
node_elements = graph.vs["node_elements"]

In [92]:
low = [0, 1, 2]
list_df = []
for t in low:
  df = dbcd_sort.iloc[node_elements[t]]
  list_df.append(df)

data_low = pd.concat(list_df)
data_low

,A1,A4,A5,A6,A7,A8,A9,A10,A11,A12,...,A4913,A4914,A4915,A4916,A4917,A4918,A4919,A4920,A4921,A4922
176,0.054812,0.019935,-0.043177,0.066085,-0.033071,0.163610,-0.175697,-0.034097,-0.057761,0.177973,...,-0.025966,0.035657,-0.102352,0.123709,-0.231527,0.175750,0.092046,-0.094414,-0.759283,0.387694
138,0.711861,0.937235,-0.244684,-0.116267,0.054862,-0.081377,0.051549,-0.083982,-0.025607,-0.678028,...,0.272174,0.141231,0.227815,-0.134530,0.509213,-0.075797,-0.105237,-0.143826,-0.803545,-0.287106
264,0.974736,-1.447581,-0.549368,-0.247172,-0.764167,-0.176743,-0.112853,0.054949,-0.076646,-0.298745,...,0.041889,0.191530,0.120646,-0.621534,0.536037,0.082599,-0.045721,-0.177401,-0.384109,-0.484176
143,1.316982,-0.303693,0.520857,-0.336211,-0.076130,-0.044480,-0.055277,-0.155827,-0.095284,-0.770644,...,0.009139,0.017278,-0.136529,-0.258067,-0.225903,-0.298038,-0.132905,-0.075359,-0.186038,-0.176791
68,1.486729,-0.178980,-0.110528,-0.288066,-0.159630,0.076776,-0.191567,-0.320597,-0.086783,-0.297414,...,-0.038455,0.278693,0.099813,0.323208,0.194249,-0.105758,-0.083864,0.162921,0.234068,0.231832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,6.789922,-0.362437,-0.249457,-0.328274,0.363052,0.710949,0.043364,0.070355,0.170909,-1.243471,...,0.109080,-0.217955,-0.159378,-0.020143,-0.212620,-0.594361,-0.154453,-0.068600,-1.082802,-0.181465
241,6.874778,-0.319027,-0.188408,-0.010666,0.162474,0.263949,0.452325,0.261698,-0.052526,-0.663169,...,0.128801,0.373921,0.084344,-0.041338,-0.133645,0.208348,-0.222919,0.001922,-0.348856,0.617375
169,7.088323,-0.397688,-0.177384,-0.305150,-0.112899,-0.012563,0.134572,-0.059352,0.443482,-1.011636,...,0.000562,-0.181546,0.081045,-0.189778,-0.277776,-0.371865,-0.152433,0.079769,-0.474483,-0.644646
292,7.378547,-0.521894,-0.280549,-0.314779,0.132494,-0.213776,0.326215,-0.309374,-0.080955,-0.256351,...,-0.538544,-0.002992,0.206624,-0.022945,-0.107030,-0.443997,0.030036,-0.205376,-0.937327,-0.441240


In [93]:
high = [8, 9, 10, 11]
list_df = []
for t in high:
  df = dbcd_sort.iloc[node_elements[t]]
  list_df.append(df)

data_high = pd.concat(list_df)
data_high

,A1,A4,A5,A6,A7,A8,A9,A10,A11,A12,...,A4913,A4914,A4915,A4916,A4917,A4918,A4919,A4920,A4921,A4922
113,10.908003,0.310934,-0.107818,-0.025042,-0.074744,-0.147670,0.010446,-0.280855,-0.195581,-0.366516,...,0.084158,0.171700,0.120991,-0.037717,0.103021,-0.151291,-0.013143,0.043567,0.321415,0.487486
82,11.017040,-0.533789,-0.228425,-0.214381,0.088544,-0.213859,0.524582,-0.146433,-0.370619,-0.349710,...,0.121606,-0.113548,-0.061404,-0.134658,-0.444872,-0.059882,0.005422,-0.132903,-1.061709,-0.188216
16,11.017054,-0.792686,-0.659530,-0.246509,0.027429,-0.242046,-0.348524,-0.219347,-0.282600,-0.176612,...,0.009693,0.095116,-0.177920,-0.175897,-0.389477,0.027270,-0.119599,-0.037314,-0.403015,-0.151956
117,11.036043,-0.265280,-0.139957,0.407562,-0.191066,-0.310489,0.052252,-0.169377,-0.098329,-0.013685,...,0.270769,0.055868,0.005729,-0.083288,-0.205401,0.264637,0.052400,0.013022,-0.007112,-0.524671
277,11.083095,0.010581,-0.005168,0.647517,-0.016755,-0.371488,-0.102141,0.307503,0.136909,-0.173547,...,-0.150619,-0.206741,-0.192285,-0.065068,-0.066656,0.069758,0.186125,-0.159033,-0.174767,-0.489927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,16.871000,-0.338551,0.116952,-0.204193,-0.140055,-0.069522,-0.229249,-0.311921,-0.054290,0.056292,...,0.046131,-0.128866,-0.308462,-0.122217,-0.197172,-0.071542,-0.186752,-0.043103,0.039062,0.304252
263,17.153042,-0.283146,-0.025771,0.065957,-0.175545,-0.099650,-0.166955,0.040650,0.045920,-0.551395,...,0.107698,0.673904,-0.105849,-0.295713,0.047346,0.174357,-0.028160,-0.132984,0.136637,-0.132075
262,17.487091,-0.001087,0.181731,0.296670,-0.190403,-0.218437,-0.065673,-0.066405,0.184229,0.059148,...,0.147208,0.044413,-0.163085,0.099709,-0.153744,0.171604,-0.010180,0.118427,0.055516,-0.581992
73,17.659092,-0.318378,-0.003260,-0.247697,-0.092754,-0.132286,-0.181909,-0.342291,-0.199297,-0.106317,...,0.040297,-0.180987,-0.162543,0.355304,-0.030736,-0.251088,-0.082230,0.051855,-0.047637,0.027934


In [15]:
data_low.to_excel('data_low.xlsx')
data_high.to_excel('data_high.xlsx')

In [5]:
data_low=pd.read_excel('/content/data_low.xlsx')
data_high=pd.read_excel('/content/data_high.xlsx')

In [111]:
t_stat = []                                       #list kosong untuk menyimpan nilai t-hitung
p_value = []                                      #list kosong untuk menyimpan p-value
sign_ = []                                        #list kosong untuk menyimpan keputusan signifikan atau tidak
sig_feature = []                                  #list kosong untuk menyimpan feature yang sedang diuji-t
data_ttest = pd.DataFrame()
for k in dbcd_sort.keys()[1:]:
  t, p = stats.ttest_ind(data_low[k], data_high[k])      #uji t
  t_stat.append(t)
  p_value.append(p)
  if p<0.000000001:                                            #aturan alpha bisa disesuaikan 0.001 dpt 32
    sig_feature.append(k)
    sign_.append('significant')
  else:
    sign_.append('not significant')


data_ttest['feature'] = dbcd_sort.keys()[1:]
data_ttest['t-stat'] = t_stat
data_ttest['p-value'] = p_value
data_ttest['decision'] = sign_
data_ttest

,feature,t-stat,p-value,decision
0,A4,-0.561012,0.575584,not significant
1,A5,-3.062117,0.002584,not significant
2,A6,-3.467446,0.000677,not significant
3,A7,2.722158,0.007214,not significant
4,A8,1.100488,0.272793,not significant
...,...,...,...,...
4914,A4918,-4.331452,0.000026,not significant
4915,A4919,-3.289145,0.001239,not significant
4916,A4920,-4.959278,0.000002,not significant
4917,A4921,-1.507767,0.133611,not significant


In [112]:
len(sig_feature)

39

In [113]:
sig_feature

['A84',
 'A216',
 'A450',
 'A471',
 'A796',
 'A908',
 'A1004',
 'A1120',
 'A1169',
 'A1470',
 'A1532',
 'A1688',
 'A1772',
 'A1880',
 'A2045',
 'A2245',
 'A2404',
 'A2803',
 'A2933',
 'A3071',
 'A3120',
 'A3148',
 'A3157',
 'A3281',
 'A3397',
 'A3650',
 'A3795',
 'A3825',
 'A4019',
 'A4104',
 'A4123',
 'A4244',
 'A4265',
 'A4377',
 'A4378',
 'A4459',
 'A4524',
 'A4580',
 'A4807']

# Survival Analysis

In [51]:
!pip install lifelines

In [52]:
!pip install scikit-survival==0.22.0

In [53]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from lifelines import LogLogisticAFTFitter, LogNormalAFTFitter, WeibullAFTFitter
from sklearn.model_selection import train_test_split
from sksurv.ensemble import RandomSurvivalForest
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#import eli5
#from eli5.sklearn import PermutationImportance
from scipy import stats
from lifelines.utils import k_fold_cross_validation
%pylab inline
%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


In [114]:
surv_important = dbcd[sig_feature]
surv_important['time'] = dbcd['A1']
surv_important['status']=dbcd['A2']

<ipython-input-114-135cf9f6b330>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-114-135cf9f6b330>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [115]:
X_train, X_test, E_train, E_test, Y_train, Y_test = train_test_split(dbcd[sig_feature], surv_important['status'], surv_important['time'], test_size=0.25, random_state=6)
aft_weibull_imp = WeibullAFTFitter()
aft_weibull_imp.fit(pd.concat([X_train, E_train, Y_train], axis=1), duration_col='time', event_col='status', robust=True)
aft_weibull_imp.print_summary()
print("C-Index for testing= ", aft_weibull_imp.score(pd.concat([X_test, E_test, Y_test], axis=1), scoring_method="concordance_index"))

<lifelines.WeibullAFTFitter: fitted with 221 total observations, 161 right-censored observations>
             duration col = 'time'
                event col = 'status'
          robust variance = True
   number of observations = 221
number of events observed = 60
           log-likelihood = -213.28
         time fit was run = 2024-07-14 09:12:12 UTC

---
                   coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
param   covariate                                                                                                  
lambda_ A1004      0.30      1.35      0.37           -0.42            1.02                0.66                2.77
        A1120      0.14      1.15      0.90           -1.62            1.89                0.20                6.63
        A1169     -0.51      0.60      0.85           -2.17            1.15                0.11                3.15
        A1470      0.55      1.74      0.73           -0.87            1.98                0.42                7.24
        A1532      0.74      2.09      0.53           -0.31            1.79                0.73                5.96
        A1688      0.02      1.02      0.88           -1.71            1.76                0.18                5.79
        A1772      0.50      1.64      0.76           -0.99            1.99                0.37                7.30
        A1880     -0.03      0.97      0.55           -1.10            1.04                0.33                2.83
        A2045      0.33      1.40      0.32           -0.29            0.96                0.75                2.60
        A216      -0.16      0.85      0.64           -1.42            1.09                0.24                2.97
        A2245      0.57      1.77      0.51           -0.43            1.57                0.65                4.82
        A2404     -0.67      0.51      0.85           -2.33            0.99                0.10                2.68
        A2803      0.88      2.42      0.88           -0.83            2.60                0.43               13.46
        A2933     -1.39      0.25      0.97           -3.29            0.52                0.04                1.68
        A3071      0.62      1.86      1.52           -2.36            3.61                0.09               36.79
        A3120     -0.59      0.56      0.45           -1.46            0.29                0.23                1.33
        A3148     -0.24      0.78      0.73           -1.68            1.19                0.19                3.29
        A3157     -0.28      0.76      0.64           -1.53            0.98                0.22                2.65
        A3281      0.57      1.78      0.61           -0.63            1.78                0.53                5.92
        A3397      0.50      1.65      0.43           -0.33            1.34                0.72                3.81
        A3650      0.28      1.32      1.26           -2.19            2.75                0.11               15.66
        A3795     -0.20      0.82      0.59           -1.36            0.97                0.26                2.63
        A3825     -0.17      0.84      0.51           -1.17            0.82                0.31                2.27
        A4019     -1.38      0.25      1.02           -3.38            0.62                0.03                1.86
        A4104     -0.20      0.82      0.86           -1.89            1.49                0.15                4.42
        A4123      0.15      1.16      0.85           -1.52            1.82                0.22                6.15
        A4244     -1.44      0.24      1.12           -3.64            0.75                0.03                2.13
        A4265     -0.66      0.52      0.58           -1.80            0.49                0.17                1.63
        A4377     -0.58      0.56      0.66           -1.88            0.71                0.15                2.04
        A4378     -0.59      0.55      0.67  

C-Index for testing=  0.8336673346693386


In [116]:
aft_loglogistic_imp = LogLogisticAFTFitter()
aft_loglogistic_imp.fit(pd.concat([X_train, E_train, Y_train], axis=1), duration_col='time', event_col='status', robust=True)
aft_loglogistic_imp.print_summary()
print("C-Index for testing= ", aft_loglogistic_imp.score(pd.concat([X_test, E_test, Y_test], axis=1), scoring_method="concordance_index"))

<lifelines.LogLogisticAFTFitter: fitted with 221 total observations, 161 right-censored observations>
             duration col = 'time'
                event col = 'status'
          robust variance = True
   number of observations = 221
number of events observed = 60
           log-likelihood = -210.07
         time fit was run = 2024-07-14 09:12:15 UTC

---
                  coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
param  covariate                                                                                                  
alpha_ A1004      0.22      1.25      0.32           -0.41            0.86                0.66                2.36
       A1120      0.41      1.50      0.88           -1.32            2.13                0.27                8.45
       A1169     -0.31      0.73      0.69           -1.68            1.05                0.19                2.85
       A1470      0.23      1.25      0.69           -1.12            1.57                0.33                4.82
       A1532      0.51      1.66      0.57           -0.62            1.63                0.54                5.11
       A1688     -0.09      0.91      0.90           -1.86            1.68                0.15                5.38
       A1772      0.54      1.72      0.53           -0.50            1.58                0.61                4.87
       A1880     -0.07      0.94      0.51           -1.07            0.94                0.34                2.56
       A2045      0.30      1.34      0.28           -0.24            0.83                0.78                2.30
       A216       0.01      1.01      0.51           -0.99            1.01                0.37                2.74
       A2245      0.39      1.47      0.43           -0.46            1.23                0.63                3.42
       A2404     -0.72      0.48      1.03           -2.74            1.29                0.06                3.64
       A2803      0.68      1.98      1.05           -1.37            2.74                0.25               15.47
       A2933     -1.26      0.28      0.92           -3.06            0.55                0.05                1.73
       A3071      0.53      1.70      1.23           -1.87            2.93                0.15               18.82
       A3120     -0.55      0.58      0.39           -1.31            0.21                0.27                1.24
       A3148     -0.25      0.78      0.67           -1.56            1.07                0.21                2.90
       A3157     -0.25      0.78      0.63           -1.50            0.99                0.22                2.70
       A3281      0.45      1.56      0.59           -0.72            1.61                0.49                5.00
       A3397      0.65      1.91      0.36           -0.06            1.35                0.94                3.86
       A3650      0.04      1.04      1.16           -2.23            2.31                0.11               10.10
       A3795     -0.17      0.84      0.62           -1.38            1.04                0.25                2.82
       A3825     -0.06      0.94      0.44           -0.93            0.81                0.40                2.26
       A4019     -1.08      0.34      0.85           -2.75            0.58                0.06                1.79
       A4104      0.41      1.50      0.83           -1.21            2.03                0.30                7.62
       A4123      0.10      1.10      0.76           -1.40            1.59                0.25                4.91
       A4244     -0.95      0.39      1.24           -3.39            1.49                0.03                4.43
       A4265     -0.70      0.50      0.52           -1.72            0.33                0.18                1.39
       A4377     -0.71      0.49      0.67           -2.01            0.60                0.13                1.82
       A4378     -0.28      0.76      0.65           -1.56            1.

C-Index for testing=  0.7975951903807615


In [117]:
aft_lognormal_imp = LogNormalAFTFitter()
aft_lognormal_imp.fit(pd.concat([X_train, E_train, Y_train], axis=1), duration_col='time', event_col='status', robust=True)
aft_lognormal_imp.print_summary()
print("C-Index for testing= ", aft_lognormal_imp.score(pd.concat([X_test, E_test, Y_test], axis=1), scoring_method="concordance_index"))

<lifelines.LogNormalAFTFitter: fitted with 221 total observations, 161 right-censored observations>
             duration col = 'time'
                event col = 'status'
          robust variance = True
   number of observations = 221
number of events observed = 60
           log-likelihood = -208.36
         time fit was run = 2024-07-14 09:12:18 UTC

---
                  coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
param  covariate                                                                                                  
mu_    A1004      0.14      1.15      0.29           -0.43            0.71                0.65                2.04
       A1120      0.40      1.50      0.76           -1.08            1.89                0.34                6.61
       A1169     -0.22      0.80      0.56           -1.31            0.88                0.27                2.40
       A1470      0.25      1.29      0.57           -0.87            1.37                0.42                3.95
       A1532      0.49      1.64      0.48           -0.46            1.44                0.63                4.23
       A1688      0.06      1.06      0.76           -1.42            1.55                0.24                4.69
       A1772      0.66      1.93      0.42           -0.16            1.48                0.85                4.40
       A1880     -0.06      0.95      0.40           -0.85            0.73                0.43                2.08
       A2045      0.31      1.36      0.23           -0.14            0.76                0.87                2.13
       A216       0.03      1.03      0.43           -0.81            0.88                0.45                2.40
       A2245      0.29      1.33      0.33           -0.35            0.93                0.70                2.53
       A2404     -0.69      0.50      0.79           -2.24            0.87                0.11                2.39
       A2803      0.78      2.18      0.82           -0.82            2.38                0.44               10.85
       A2933     -1.09      0.34      0.71           -2.48            0.30                0.08                1.36
       A3071      0.48      1.62      0.93           -1.35            2.31                0.26               10.07
       A3120     -0.48      0.62      0.34           -1.15            0.18                0.32                1.20
       A3148     -0.11      0.89      0.53           -1.14            0.92                0.32                2.50
       A3157     -0.34      0.71      0.47           -1.25            0.57                0.29                1.77
       A3281      0.36      1.43      0.50           -0.61            1.33                0.54                3.79
       A3397      0.67      1.96      0.30            0.08            1.27                1.09                3.55
       A3650      0.10      1.11      1.02           -1.90            2.11                0.15                8.23
       A3795     -0.17      0.85      0.51           -1.16            0.83                0.31                2.30
       A3825     -0.17      0.84      0.37           -0.91            0.56                0.40                1.75
       A4019     -1.12      0.32      0.73           -2.56            0.31                0.08                1.36
       A4104      0.39      1.48      0.74           -1.06            1.84                0.35                6.29
       A4123      0.05      1.05      0.67           -1.27            1.36                0.28                3.89
       A4244     -0.76      0.47      1.04           -2.81            1.28                0.06                3.60
       A4265     -0.73      0.48      0.44           -1.59            0.13                0.20                1.14
       A4377     -0.62      0.54      0.56           -1.71            0.47                0.18                1.60
       A4378     -0.21      0.81      0.59           -1.36            0.94

C-Index for testing=  0.8036072144288577


In [118]:
#Buat label RSF
dt = [('cens', '?'), ('time', '<f8')]
Y_ = []
for i, a in enumerate(surv_important['time']):
  if surv_important['status'][i]==1:
    Y_.append((True, a))
  else:
    Y_.append((False, a))

Y_ar = np. array(Y_, dtype=dt)
Y_ar

array([(False, 12.9970417), (False, 11.1570836), (False, 10.1380805),
       (False,  8.8022009), (False, 10.2940645), (False,  5.8042112),
       (False,  7.8576216), (False,  8.1670909), (False,  8.2327145),
       (False,  7.8658207), (False,  6.9706851), (False,  5.1855713),
       (False,  6.245008 ), (False, 11.3890819), (False, 10.1080349),
       (False,  7.3539163), (False, 11.017054 ), (False,  4.7315923),
       ( True,  2.1726437), ( True,  9.526054 ), ( True,  8.4658698),
       ( True,  5.1508002), ( True,  5.3562837), (False,  4.9946889),
       ( True,  7.928802 ), (False,  4.1178098), ( True,  2.7096374),
       ( True,  2.6083967), ( True,  5.504162 ), ( True,  2.619267 ),
       ( True,  2.2905456), ( True,  3.7370838), ( True,  5.7796823),
       ( True,  3.4552542), (False,  3.2252092), (False,  2.3107295),
       ( True,  3.2526602), ( True,  3.2416183), (False,  5.3032733),
       (False,  5.2320549), (False, 10.0970156), (False, 14.8170925),
       (False, 14.26

In [119]:
X_train, X_test, Y_train, Y_test = train_test_split(surv_important[sig_feature], Y_ar, test_size=0.25, random_state=6)
rsf_dbcd = RandomSurvivalForest(n_estimators=70,
                           min_samples_split=3,
                           min_samples_leaf=10,
                           max_features="sqrt",
                           n_jobs=-1, random_state=1)
rsf_dbcd.fit(X_train, Y_train)
c_rsf1 = rsf_dbcd.score(X_test, Y_test)
print("C-Index = ", c_rsf1)

C-Index =  0.7925851703406813
